In [1]:
#data loading
import pandas as pd

predata = pd.read_csv(r'C:/Users/BeatrizCarvalho/OneDrive - Closer Consultoria Lda/Documents/Entangled-Spaces/Transfers/Datasets/0transfers_nd_corr.csv')

C:\Users\BeatrizCarvalho\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
#imbalance of the dataset
predata['is_fraud'].value_counts()

0    500000
1       569
Name: is_fraud, dtype: int64

In [6]:
#initial number of columns and rows of the dataframe
predata.shape

(500569, 19)

In [5]:
#visualize the whole output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
#ignore warnings from ctgan
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [ ]:
#generate synthetic frauds  
from sdv.tabular import CTGAN

model = CTGAN(epochs = 100, verbose = True) 

model.fit(predata) 

In [ ]:
#save the trained model
model.save(r'C:\Users\BeatrizCarvalho\OneDrive - Closer Consultoria Lda\Documents\Entangled-Spaces\Transfers\Datasets\gan.pkl')

In [ ]:
#number of frauds to generate 
f = len(predata[(predata['is_fraud'] == 0)]) - len(predata[(predata['is_fraud'] == 1)])

In [ ]:
#conditional sampling
from sdv.sampling import Condition

#generate only values that satisfy this condition
condition = Condition({'is_fraud': 1}, num_rows = f)

#generate synthetic data
frauds_data = model.sample_conditions(conditions = [condition])

In [ ]:
#create the new dataframe
predatabalanced = pd.concat([predata, frauds_data], ignore_index = True) 

In [ ]:
#make sure the dataset is now balanced
predatabalanced['is_fraud'].value_counts()

In [ ]:
#comparision of the shape of the actual data and the generated data
from table_evaluator import load_data, TableEvaluator

predata.shape, predatabalanced.shape

In [ ]:
#evaluate the synthetic data
from sdv.evaluation import evaluate

#separate scores 
evaluate(predatabalanced, predata, metrics = ['CSTest', 'KSTest', 'GMLogLikelihood'], aggregate = False)

#CSTest - chi-2 test to compare the distributions of the discrete (categorical/boolean) columns returns the average of the 
#p-values obtained across all the columns
#KSTest - 2 sample kolmogorov–smirnov test to compare the distributions of numerical columns using the empirical CDF returns 
#the average of the KS statistic values obtained across all the columns
#GMLogLikelihood - fits GMM to the real data and evaluates how likely the synthetic data belongs to the same distribution as 
#the real data

In [ ]:
#evaluate how similar the two dataframes are by graphics 
table_evaluator =  TableEvaluator(predata, predatabalanced) 

table_evaluator.visual_evaluation()

In [ ]:
#save the balanced dataset
predatabalanced.to_csv(r'C:/Users/BeatrizCarvalho/OneDrive - Closer Consultoria Lda/Documents/Entangled-Spaces/Datasets/2transfers_balanced_gans+rund.csv', index = False)